In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras import layers, models, Input, Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

#loading the pre processed data
train_df = pd.read_csv('train_preprocessed.csv')

#scaling the price_log to be between 0 and 1 to prevent high losses
target_scaler = MinMaxScaler()
train_df['target_scaled'] = target_scaler.fit_transform(train_df[['price_log']])

#splitting the data
train_split, val_split = train_test_split(train_df, test_size=0.2, random_state=42)

In [2]:
def multi_modal_generator(df, image_folder, batch_size=32, target_size=(224, 224), shuffle=True):
    while True:
        if shuffle: df = df.sample(frac=1).reset_index(drop=True)
        for i in range(0, len(df), batch_size):
            batch_df = df.iloc[i:i+batch_size]
            images, tabular, labels = [], [], []
            for _, row in batch_df.iterrows():
                img_path = os.path.join(image_folder, f"{int(row['id'])}.jpg")
                if os.path.exists(img_path):
                    img = img_to_array(load_img(img_path, target_size=target_size)) / 255.0
                    tab_feat = row.drop(['id', 'price', 'price_log', 'target_scaled']).values.astype(np.float32)
                    images.append(img)
                    tabular.append(tab_feat)
                    labels.append(row['target_scaled']) 
            if images:
                yield ((np.array(images), np.array(tabular)), np.array(labels))

output_sig = (
    (tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32), 
     tf.TensorSpec(shape=(None, 15), dtype=tf.float32)),
    tf.TensorSpec(shape=(None,), dtype=tf.float32)
)

In [3]:
#creating the brnaches of the CNN
img_in = Input(shape=(224, 224, 3))
x = layers.Conv2D(16, (3,3), activation='relu')(img_in)
x = layers.MaxPooling2D((2,2))(x)
x = layers.GlobalAveragePooling2D()(x)

tab_in = Input(shape=(15,))
y = layers.Dense(64, activation='relu')(tab_in)
y = layers.Dense(32, activation='relu')(y)

#fusing the branches
merged = layers.concatenate([x, y])
z = layers.Dense(16, activation='relu')(merged)

#using the sigmoid function to ensure the output is between 0 and 1
output = layers.Dense(1, activation='sigmoid')(z) 

model = Model(inputs=[img_in, tab_in], outputs=output)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse')

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 222, 222,  │        448 │ input_layer[0][0] │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 15)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 111, 111,  │          0 │ conv2d[0][0]      │
│ (MaxPooling2D)      │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │      1,024 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 16)        │          0 │ max_pooling2d[0]… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 32)        │      2,080 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 48)        │          0 │ global_average_p… │
│ (Concatenate)       │                   │            │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 16)        │        784 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 1)         │         17 │ dense_2[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 4,353 (17.00 KB)

 Trainable params: 4,353 (17.00 KB)

 Non-trainable params: 0 (0.00 B)

In [4]:
train_ds = tf.data.Dataset.from_generator(
    lambda: multi_modal_generator(train_split, 'images/train'), output_signature=output_sig)
val_ds = tf.data.Dataset.from_generator(
    lambda: multi_modal_generator(val_split, 'images/train', shuffle=False), output_signature=output_sig)

history = model.fit(train_ds, steps_per_epoch=len(train_split)//32, 
                    validation_data=val_ds, validation_steps=len(val_split)//32, 
                    epochs=20)

Epoch 1/20
405/405 ━━━━━━━━━━━━━━━━━━━━ 96s 233ms/step - loss: 0.0103 - val_loss: 0.0050
Epoch 2/20
405/405 ━━━━━━━━━━━━━━━━━━━━ 78s 192ms/step - loss: 0.0050 - val_loss: 0.0047
Epoch 3/20
405/405 ━━━━━━━━━━━━━━━━━━━━ 79s 194ms/step - loss: 0.0044 - val_loss: 0.0042
Epoch 4/20
405/405 ━━━━━━━━━━━━━━━━━━━━ 78s 194ms/step - loss: 0.0042 - val_loss: 0.0043
Epoch 5/20
405/405 ━━━━━━━━━━━━━━━━━━━━ 76s 188ms/step - loss: 0.0040 - val_loss: 0.0039
Epoch 6/20
405/405 ━━━━━━━━━━━━━━━━━━━━ 77s 190ms/step - loss: 0.0039 - val_loss: 0.0040
Epoch 7/20
405/405 ━━━━━━━━━━━━━━━━━━━━ 73s 181ms/step - loss: 0.0039 - val_loss: 0.0042
Epoch 8/20
405/405 ━━━━━━━━━━━━━━━━━━━━ 70s 172ms/step - loss: 0.0038 - val_loss: 0.0039
Epoch 9/20
405/405 ━━━━━━━━━━━━━━━━━━━━ 70s 174ms/step - loss: 0.0037 - val_loss: 0.0037
Epoch 10/20
405/405 ━━━━━━━━━━━━━━━━━━━━ 71s 174ms/step - loss: 0.0038 - val_loss: 0.0036
Epoch 11/20
405/405 ━━━━━━━━━━━━━━━━━━━━ 70s 173ms/step - loss: 0.0036 - val_loss: 0.0036
Epoch 12/20
405/405

In [13]:
from sklearn.metrics import r2_score, mean_squared_error
import math

#creating batches of 64 for faster and efficient calculations
BATCH_SIZE = 64
target_size = (224, 224)

def get_model_predictions(df, image_folder):
    all_preds_scaled = []
    num_batches = math.ceil(len(df) / BATCH_SIZE)
        
    for i in range(num_batches):
        start = i * BATCH_SIZE
        end = min((i + 1) * BATCH_SIZE, len(df))
        batch_df = df.iloc[start:end]
        
        batch_images = []
        batch_tabular = []
        
        for _, row in batch_df.iterrows():
            img_path = os.path.join(image_folder, f"{int(row['id'])}.jpg")
            img = img_to_array(load_img(img_path, target_size=target_size)) / 255.0
            to_drop = ['id', 'price', 'price_log', 'target_scaled']
            tab_feat = row.drop(labels=to_drop, errors='ignore').values.astype(np.float32)
            
            batch_images.append(img)
            batch_tabular.append(tab_feat)
        
        preds = model.predict([np.array(batch_images), np.array(batch_tabular)], verbose=0)
        all_preds_scaled.extend(preds)
            
    return np.array(all_preds_scaled)

#transforming data to original and calculating metrics
y_val_pred_scaled = get_model_predictions(val_split, 'images/train')

y_val_pred_log = target_scaler.inverse_transform(y_val_pred_scaled)
y_val_true_log = val_split['price_log'].values.reshape(-1, 1)

y_val_true_dollars = np.expm1(y_val_true_log).flatten()
y_val_pred_dollars = np.expm1(y_val_pred_log).flatten()

r2 = r2_score(y_val_true_log, y_val_pred_log)
rmse = np.sqrt(mean_squared_error(y_val_true_dollars, y_val_pred_dollars))


print(f"Validation R^2 Score: {r2:.4f}")
print(f"Validation RMSE:      ${rmse:,.2f}")


#running same logiic with actual test data
test_df = pd.read_csv('test_preprocessed.csv')
y_test_pred_scaled = get_model_predictions(test_df, 'images/test')

# Inverse transform back to real dollar prices
y_test_pred_log = target_scaler.inverse_transform(y_test_pred_scaled)
y_test_pred_dollars = np.expm1(y_test_pred_log).flatten()

# Create and Save the Final CSV
submission_df = pd.DataFrame({
    'id': test_df['id'].astype(int),
    'predicted_price': y_test_pred_dollars
})

submission_df.to_csv('predictions.csv', index=False)
print("\n 'predictions.csv' has been created.")
display(submission_df.head())

Validation R^2 Score: 0.7106
Validation RMSE:      $177,490.47

 'predictions.csv' has been created.


,id,predicted_price
0,-1703146986,4.243701e+05
1,-615733772,7.842220e+05
2,-888484482,1.116835e+06
3,932365418,1.652684e+06
4,920926548,6.028534e+05
